In [ ]:
# Special functions and optimizations
from typing import Callable, Union, Tuple
from scipy.spatial.distance import cdist  # For space and time distance
from scipy.special import gamma, kv  # Bessel function and gamma function

import pandas as pd
import numpy as np

# Create a DataFrame with 3 columns: longitude, latitude, and time
data = {
    'longitude': np.random.uniform(-180, 180, 100),
    'latitude': np.random.uniform(-90, 90, 100),
    'time': np.random.uniform(90, 180, 100)
}

df = pd.DataFrame(data)
print(df)

df = df.to_numpy()

     longitude   latitude        time
0   129.655923  46.689523   90.778477
1    79.655502 -54.845567   95.241837
2    87.920711  -6.189835  159.012248
3     2.110736 -21.670644  170.155919
4    51.426323 -88.417881  129.341152
..         ...        ...         ...
95  171.167670  17.048809   96.141250
96  165.080400  58.210482   97.909248
97   38.734410  31.658843  174.550206
98 -117.341740  80.667345  162.044650
99 -125.756178 -16.674399  174.826960

[100 rows x 3 columns]


In [5]:
smooth = 0.5
params = [20,8,8,0.5,0.5,0.1]

range_lon, range_lat = params[1], params[2]
sqrt_range_mat = np.diag([ 1/range_lon**0.5, 1/range_lat**0.5])
sqrt_range_mat = sqrt_range_mat


# Custom distance function for cdist
def custom_distance(u, v):
    d = np.dot(sqrt_range_mat, u[:2] - v[:2] ) # Distance between x1,x2 (2D)
    spatial_diff = np.linalg.norm(d)  # Distance between x1,x2 (2D)
    temporal_diff = np.abs(u[2] - v[2])           # Distance between y1 and y2
    return np.sqrt(spatial_diff**2 + temporal_diff**2)


def matern_cov_yx_test(params: Tuple[float,float,float,float,float,float], y: np.ndarray, x: np.ndarray) -> np.ndarray:

    sigmasq, range_lat, range_lon, advec, beta, nugget  = params
    # Validate inputs
    if y is None or x is None:
        raise ValueError("Both y and x_df must be provided.")
    # Extract values
    x1 = x[:, 0]
    y1 = x[:, 1]
    t1 = x[:, 2]

    x2 = y[:, 0]
    y2 = y[:, 1]
    t2 = y[:, 2] # hour

    spat_coord1 = np.stack((x1- advec*t1, y1 - advec*t1), axis=-1)
    spat_coord2 = np.stack((x2- advec*t2, y2 - advec*t2), axis=-1)

    coords1 = np.hstack ((spat_coord1, (beta * t1).reshape(-1,1) ))
    coords2 = np.hstack ((spat_coord2, (beta * t2).reshape(-1,1) ))



    distance = cdist(coords1,coords2, metric = custom_distance)

    # Initialize the covariance matrix with zeros
    out = distance
    
    # Compute the covariance for non-zero distances

    # Compute the covariance for non-zero distances
    non_zero_indices = distance != 0
    if np.any(non_zero_indices):
        out[non_zero_indices] = (sigmasq * (2**(1-smooth)) / gamma(smooth) *
                                (distance[non_zero_indices])**smooth *
                                kv(smooth, distance[non_zero_indices]))
    out[~non_zero_indices] = sigmasq

    # Add a small jitter term to the diagonal for numerical stability
    out += np.eye(out.shape[0]) * nugget
    return out


b=matern_cov_yx_test(params,df,df)

def matern2(params: Tuple[float,float,float,float,float,float], y: np.ndarray, x: np.ndarray) -> np.ndarray:

    sigmasq, range_lat, range_lon, advec, beta, nugget  = params
    # Validate inputs
    if y is None or x is None:
        raise ValueError("Both y and x_df must be provided.")
    # Extract values
    x1 = x[:, 0]
    y1 = x[:, 1]
    t1 = x[:, 2]

    x2 = y[:, 0]
    y2 = y[:, 1]
    t2 = y[:, 2] # hour

    spat_coord1 = np.stack((x1- advec*t1, y1 - advec*t1), axis=-1)
    spat_coord2 = np.stack((x2- advec*t2, y2 - advec*t2), axis=-1)

    coords1 = np.hstack ((spat_coord1, (beta * t1).reshape(-1,1) ))
    coords2 = np.hstack ((spat_coord2, (beta * t2).reshape(-1,1) ))


    distance = cdist(coords1,coords2, metric = custom_distance)

    # Initialize the covariance matrix with zeros
    out = distance
    
    # Compute the covariance for non-zero distances

    # Compute the covariance for non-zero distances
    non_zero_indices = distance != 0
    if np.any(non_zero_indices):
        out[non_zero_indices] = sigmasq* np.exp(-distance[non_zero_indices])
    out[~non_zero_indices] = sigmasq
    

    # Add a small jitter term to the diagonal for numerical stability
    out += np.eye(out.shape[0]) * nugget
    return out
a = matern2(params,df,df)

print(b)

[[2.01000000e+01 2.73571365e-17 1.58788157e-22 ... 4.08318086e-28
  1.68558752e-45 3.35225064e-51]
 [2.73571365e-17 2.01000000e+01 5.70996767e-14 ... 8.11106007e-22
  9.45831430e-41 7.46972992e-41]
 [1.58788157e-22 5.70996767e-14 2.01000000e+01 ... 7.10172119e-10
  8.84759249e-34 9.73448239e-34]
 ...
 [4.08318086e-28 8.11106007e-22 7.10172119e-10 ... 2.01000000e+01
  4.27888597e-24 8.91042205e-26]
 [1.68558752e-45 9.45831430e-41 8.84759249e-34 ... 4.27888597e-24
  2.01000000e+01 9.42205841e-16]
 [3.35225064e-51 7.46972992e-41 9.73448239e-34 ... 8.91042205e-26
  9.42205841e-16 2.01000000e+01]]


In [6]:
np.sum(a-b)

np.float64(-5.496524335262268e-15)

In [7]:
import math

def matern_cov(d,v):
    abs_d = np.abs(d)
    if abs_d ==0:
        return 1
    else:
        sigmasq = 2
        range = 1
        out = sigmasq * (2**(1-v))/math.gamma(v) * (abs_d/range)**(v)*kv(v, abs_d/range)        
        return out  
    
print(matern_cov(4,1.5))
d = 4
abs_d = np.abs(d)
sigmasq = 2
range_ = 1
out = sigmasq * (1+ abs_d/range_)* np.exp(-abs_d/range_)
print(out)



0.18315638888734181
0.1831563888873418
